# Analisi dataset tranding di Youtube


Segue l'analisi dei dati di trending di youtube pendendo spunto da questo dataset: https://www.kaggle.com/datasnaek/youtube-new e studiando il codice del suo creatore: https://github.com/mitchelljy/Trending-YouTube-Scraper  
questo tizio mi sembra ci abbia fatto sopra un bel lavoro: https://www.kaggle.com/donyoe/exploring-youtube-trending-statistics-eda  <br>
le V che possiamo sfruttare sono: 
- __Velocity__ (perchè sono dati presi giornalmente quindi real time)
- __Variety__ (ci sono anche dei file json e poi si potrebbe integrare con altri dati) 
- volendo si può pensare anche a __Volume__ il dataset su kaggle è mezzo giga e sono 6 mesi di rilevazioni su una decina di paesi (ma la vedo complessa)

## NOTA BENE

- Non si può (almeno credo) prendere dati di trending di altri giorni se non lo stesso giorno che si fa lo scraping
- Manca la fusione con i json per ottenere la categoria!!!
- Manca la popolazione dei paesi

## Analisi tranding UK

### Punti tenuti in considerazione

- Video in trending più _vecchio_ pubblicato
- Video con _visualizzazioni likes dislikes commenti_ maggiori
- Media di _visualizzazioni likes dislikes commenti_ del trending delle giornata
- _Categoria_ che riscuote maggior successo (quindi maggior numero di video in quella categoria e confronto con gli altri paesi)
- Popolazione dei paesi rispetto ai vari indicatori
- Analizzare in base ai gruppi di età

In [2]:
import pandas as pd

### Automatizzazione della lettura

In [136]:
code = []
f = open('country_codes.txt')
for x in f:
    x = x.strip('\n')
    code.append(x)

In [169]:
# Inserire la data da considerare
day = 22
month = 12

In [167]:
dataframes = []
for i in code:
    file = "output_" + str(day)+"-" + str(month) + "-19/19." + str(day)+ "." + str(month) + "_" + str(i) + "_videos.csv"
    df = pd.read_csv(file)
    df['country_code'] = i
    dataframes.append(df)
YoutubeTrend = pd.concat(dataframes, axis = 0)
YoutubeTrend = YoutubeTrend.reset_index()

Conservo questo box sotto in caso di utilità

In [135]:
import os
import re
c = []
for filename in os.listdir(r'/home/marco/Scrivania/Università/Youtube/Youtube_trending_analysis/output_22-12-19'):
    m = re.search('_([A-Z]{2})_', filename)
    c.append(m.group(1))
c

['DE', 'IT', 'GB', 'MX', 'KR', 'JP', 'IN', 'US', 'RU', 'BR', 'CA', 'FR']

#### Video in trending più _vecchio_ pubblicato in assoluto

In [168]:
YoutubeTrend['publishedAt'] = pd.to_datetime(YoutubeTrend['publishedAt'])
YoutubeTrend['publishedAt'].sort_values()

159    2019-11-18 18:15:01+00:00
832    2019-11-20 03:59:48+00:00
2036   2019-11-22 13:05:32+00:00
1052   2019-11-22 13:05:32+00:00
350    2019-11-22 13:05:32+00:00
                  ...           
1210   2019-12-22 01:45:58+00:00
1192   2019-12-22 02:47:51+00:00
1184   2019-12-22 03:19:11+00:00
1177   2019-12-22 04:07:06+00:00
432    2019-12-22 04:30:22+00:00
Name: publishedAt, Length: 2322, dtype: datetime64[ns, UTC]

In [147]:
YoutubeTrend.loc[159][['title','publishedAt','view_count','likes','dislikes','comment_count','country_code']]

title            How to get the best fit and sound with your Po...
publishedAt                              2019-11-18 18:15:01+00:00
view_count                                                   78941
likes                                                          662
dislikes                                                       169
comment_count                                                    0
country_code                                                    US
Name: 159, dtype: object

#### Video in trending più _vecchio_ pubblicato relativo ai paesi

In [152]:
m = YoutubeTrend.groupby('country_code')['publishedAt'].idxmin()
#YoutubeTrend.loc[m]

#### Video con _visualizzazioni likes dislikes commenti_ maggiori

In [154]:
m = YoutubeTrend[['view_count','likes','dislikes','comment_count']].idxmax()
#YoutubeTrend.loc[m]

#### Video con _visualizzazioni likes dislikes commenti_ maggiori per paese

In [158]:
m = YoutubeTrend.groupby('country_code')['view_count'].idxmax()
#YoutubeTrend.loc[m]

#### Media di _visualizzazioni likes dislikes commenti_ del trending delle giornata

In [159]:
YoutubeTrend[['view_count','likes','dislikes','comment_count']].mean()

view_count       1.368667e+06
likes            6.640778e+04
dislikes         5.322524e+03
comment_count    5.680503e+03
dtype: float64

In [160]:
YoutubeTrend.groupby('country_code')[['view_count','likes','dislikes','comment_count']].mean()

,view_count,likes,dislikes,comment_count
country_code,,,,
BR,6.796129e+05,56745.560000,1076.990000,2922.710000
CA,2.151074e+06,106683.320000,2606.980000,9592.915000
DE,1.383706e+06,73405.085000,2362.435000,6435.355000
FR,1.225200e+06,62935.610000,1302.300000,4499.525000
GB,2.051334e+06,92747.640000,2210.995000,7733.765000
IN,1.933089e+06,65321.310559,3246.788820,4038.105590
IT,8.788939e+05,39041.700000,1055.465000,3442.470000
JP,1.163384e+06,32294.943590,41250.502564,6784.035897
KR,4.171489e+05,14967.054217,338.614458,1457.433735


#### _Categoria_ che riscuote maggior successo

le categorie sono indicate da numeri, il corrispettivo in testo è dato da dei file in json, per ora sto guardando quelli del tizio su kaggle però ci sarà un modo per trovarli anche direttamente tramite API. Questa cosa è molto figa perchè possiamo dire di utilizzare 2 tipi di dato, uno csv e l'altro json (anche se il json è semplice quindi magari meglio trovare altre fonti da unire a questi dati ahaha)

In [4]:
df.groupby('categoryId').count()['video_id']

categoryId
1      7
2      3
10    20
15     1
17    30
19     1
20    12
22    14
23    17
24    68
25     8
26     6
27     6
28     7
Name: video_id, dtype: int64

In [26]:
pop = pd.read_csv('pop.csv')
pop = pop.transpose()
pop = pop.reset_index()
#pop = pop.drop(['Index','0'],axis=1)
pop

,index,0,1,2,3,4,5,6,7,8,...,258,259,260,261,262,263,264,265,266,267
0,Unnamed: 0,NaN,NaN,NaN,Country Name,Aruba,Afghanistan,Angola,Albania,Andorra,...,Virgin Islands (U.S.),Vietnam,Vanuatu,World,Samoa,Kosovo,"Yemen, Rep.",South Africa,Zambia,Zimbabwe
1,Unnamed: 1,NaN,NaN,NaN,Country Code,ABW,AFG,AGO,ALB,AND,...,VIR,VNM,VUT,WLD,WSM,XKX,YEM,ZAF,ZMB,ZWE
2,Unnamed: 2,NaN,NaN,NaN,Indicator Name,"Population, total","Population, total","Population, total","Population, total","Population, total",...,"Population, total","Population, total","Population, total","Population, total","Population, total","Population, total","Population, total","Population, total","Population, total","Population, total"
3,Unnamed: 3,NaN,NaN,NaN,Indicator Code,SP.POP.TOTL,SP.POP.TOTL,SP.POP.TOTL,SP.POP.TOTL,SP.POP.TOTL,...,SP.POP.TOTL,SP.POP.TOTL,SP.POP.TOTL,SP.POP.TOTL,SP.POP.TOTL,SP.POP.TOTL,SP.POP.TOTL,SP.POP.TOTL,SP.POP.TOTL,SP.POP.TOTL
4,Unnamed: 4,NaN,NaN,NaN,1960,54211,8.99697e+06,5.45493e+06,1.6088e+06,13411,...,32500,3.267e+07,63689,3.03202e+09,108629,947000,5.31536e+06,1.70998e+07,3.07078e+06,3.77668e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Unnamed: 59,NaN,NaN,NaN,2015,104341,3.44136e+07,2.78844e+07,2.8807e+06,78011,...,107710,9.26771e+07,271130,7.34055e+09,193513,1.8018e+06,2.64979e+07,5.53864e+07,1.58794e+07,1.38146e+07
60,Unnamed: 60,NaN,NaN,NaN,2016,104872,3.53831e+07,2.88425e+07,2.8761e+06,77297,...,107510,9.36387e+07,278330,7.4261e+09,194535,1.8162e+06,2.71682e+07,5.62037e+07,1.63635e+07,1.40304e+07
61,Unnamed: 61,NaN,NaN,NaN,2017,105366,3.62964e+07,2.98167e+07,2.87346e+06,77001,...,107268,9.45966e+07,285510,7.51099e+09,195352,1.8307e+06,2.78348e+07,5.70005e+07,1.68537e+07,1.42367e+07
62,Unnamed: 62,NaN,NaN,NaN,2018,105845,3.71724e+07,3.08098e+07,2.86638e+06,77006,...,106977,9.55404e+07,292680,7.59427e+09,196130,1.8453e+06,2.84987e+07,5.77796e+07,1.73518e+07,1.4439e+07
